In [1]:
import random
import itertools
from pathlib import Path

import torch
import numpy as np

import params
from common import AnalyticRetardation, ConcentrationPredictor

In [2]:
cfg = params.Parameters()
WRITE_DATA = True

In [3]:
def interpolate(y, xmin, xmax, x):
    y = y.reshape(-1)
    x = x.reshape(-1)

    n = len(y) - 1  # number of intervals
    x_points = torch.linspace(xmin, xmax, n + 1)
    dx = (xmax - xmin) / n

    # Calculate the index of the interval
    i = torch.clip(((x - xmin) / dx).to(int), 0, n - 1)

    # Perform linear interpolation using broadcasting
    y_interp = y[i] + (y[i + 1] - y[i]) * (x - x_points[i]) / dx

    return y_interp.reshape(-1, 1, 1)

In [4]:
def mse(a, b):
    return np.mean((a - b) ** 2)


def max_abs_err(a, b):
    return np.max(np.abs(a - b))

In [5]:
seed = 12345
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
random.seed(seed)
np.random.seed(seed)

ret_fun_name = "langmuir"
# ret_fun_name = "../data_out/FINN_forward_tests/finn_langmuir_github_c/predicted_retardations/retPred_100.npy"


def get_ret_inv_fun():
    if ret_fun_name == "freundlich":

        def fun(c):
            return AnalyticRetardation.freundlich(
                c, por=cfg.por, rho_s=cfg.rho_s, Kf=cfg.Kf, nf=cfg.nf
            )
    elif ret_fun_name == "linear":

        def fun(c):
            return AnalyticRetardation.linear(
                c, por=cfg.por, rho_s=cfg.rho_s, Kd=cfg.Kd
            )
    elif ret_fun_name == "langmuir":

        def fun(c):
            return AnalyticRetardation.langmuir(
                c, por=cfg.por, rho_s=cfg.rho_s, smax=cfg.smax, Kl=cfg.Kl
            )
    else:

        def fun(c):
            return interpolate(torch.from_numpy(np.load(ret_fun_name)), 0, 1, c)

    return lambda c: 1.0 / fun(c)


ret_inv_fun = get_ret_inv_fun()

u0 = torch.zeros(size=(2, cfg.Nx, 1))
model = ConcentrationPredictor(
    u0=u0,
    cfg=cfg,
    ret_inv_funs=[ret_inv_fun, None],
)

t = torch.linspace(0.0, cfg.T, cfg.Nt)
print("Computing...")
model.eval()
c = model(t)
c = c.detach().numpy()
print("Finished computing")
print(c.shape)

Computing...
Finished computing
(2001, 2, 26, 1)


In [6]:
u_ret = torch.linspace(0.0, 1.0, 100)
ret_from_model = model.retardation(u_ret).detach().numpy()
ret_from_function = (1.0 / ret_inv_fun(u_ret)).detach().numpy()

print(f"Ret. Max abs error: {max_abs_err(ret_from_model, ret_from_function):.2e}")
print(f"Ret. MSE: {mse(ret_from_model, ret_from_function):.2e}")

Ret. Max abs error: 0.00e+00
Ret. MSE: 0.00e+00


In [7]:
assert np.allclose(ret_from_model, ret_from_function)

out_path = Path(f"../data/FINN_forward_solver/retardation_{ret_fun_name}/c_train.npy")
out_path.parent.mkdir(parents=True, exist_ok=True)
if WRITE_DATA:
    print(f"Saving to {out_path}")
    np.save(out_path, c[..., 0])  # remove the last dimension

In [9]:
c_train = np.load("../data_out_github_c/FINN_forward_tests/finn_langmuir_github_c/c_train.npy")
c_pred = np.load(
    "../data_out_github_c/FINN_forward_tests/finn_langmuir_github_c/c_full_predictions.npy"
)
c_langmuir_github = np.load("../data/synthetic_data/retardation_langmuir/c_train.npy")[
    :c_train.shape[0], ..., None
]
c_langmuir_FINN = np.load(
    "../data/FINN_forward_solver/retardation_langmuir/c_train.npy"
)[:c_train.shape[0], ..., None]

print(c_train.shape)
print(c_pred.shape)
print(c_langmuir_github.shape)
print(c_langmuir_FINN.shape)

(51, 2, 26, 1)
(51, 2, 26, 1)
(51, 2, 26, 1)
(51, 2, 26, 1)


Was ich weiß:
- MSE(c_pred, c_train) ist klein.
- c_train == c_langmuir_github

Was ich wissen will:
- [x] Ist c == c_pred? (Dann ist auch mse(c, c_train) == mse(c_pred, c_train))
    - Leider nein. Das heißt, dass da irgendwas falsch läuft beim FINN forward code.
        - [x] Ist es die Interpolation?
            - Habe ich getestet, indem ich langmuir auch mit linearer Interpolation berechne und den Unterschied zwischen dem normalen anschaue. Die Antwort ist: MSE ist kleiner als 1e-11, max abs err < 1e-5. Fazit: Es ist nicht die Interpolation.

In [10]:
cs = {
    "c": c[:c_train.shape[0], ...],
    "c_train": c_train,
    "c_pred": c_pred,
    "c_langmuir_github": c_langmuir_github,
    "c_langmuir_FINN": c_langmuir_FINN,
}
# iter pairwise, compute mse and print names
for i, j in itertools.combinations(cs.keys(), 2):
    print(
        f"{i:>20} | {j:<20}: {mse(cs[i], cs[j]):.2e}, {max_abs_err(cs[i], cs[j]):.2e}"
    )

                   c | c_train             : 1.80e-01, 1.97e+00
                   c | c_pred              : 1.80e-01, 1.97e+00
                   c | c_langmuir_github   : 1.80e-01, 1.97e+00
                   c | c_langmuir_FINN     : 0.00e+00, 0.00e+00
             c_train | c_pred              : 7.38e-07, 4.16e-03
             c_train | c_langmuir_github   : 2.93e-17, 2.97e-08
             c_train | c_langmuir_FINN     : 1.80e-01, 1.97e+00
              c_pred | c_langmuir_github   : 7.38e-07, 4.16e-03
              c_pred | c_langmuir_FINN     : 1.80e-01, 1.97e+00
   c_langmuir_github | c_langmuir_FINN     : 1.80e-01, 1.97e+00
